In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq self-supervised

In [ ]:
# default_exp vision.metrics

# Vision.Metrics

> Metrics for tracking performance of self-supervised training. It aims to give an idea about the quality of the learned representations during training in the presence of a labeled validation set. It can be used to decide how much longer to keep training. Since self-supervised models usually favor more epochs it can help save time and computation.

In [ ]:
#export
from fastai.vision.all import *

In [ ]:
#export
class KNNProxyMetric(Callback):
    "A metric which calculates knn-1 accuracy. Use with a labeled validation set."
    order,run_train,run_valid=8,False,True
            
    def before_batch(self):
        self.orig_x, self.orig_y = self.x, self.y
    
    def before_validate(self):
        self.embs = tensor([]).to(self.dls.device)
        self.targs = tensor([]).to(self.dls.device)
        
    def after_pred(self):
        self.embs = torch.cat([self.embs, self.model.encoder(self.orig_x)])
        self.targs = torch.cat([self.targs, self.orig_y])
  
    def accuracy(self): 
        self.embs = F.normalize(self.embs)
        sim = self.embs @ self.embs.T
        nearest_neighbor = sim.argsort(dim=1, descending=True)[:,1]
        return (self.targs == self.targs[nearest_neighbor]).float().mean()
        
    def after_fit(self):
        del self.embs, self.targs
        torch.cuda.empty_cache()

In [ ]:
#TODO: Add a test to check accuracy

### Example Usage

In [ ]:
from self_supervised.layers import *
from self_supervised.vision.simclr import *

Create your dataset as usual. Make sure your validation set has labels. For example, if your dataset has 10% of labeled data and 90% of unlabeled data you can use that 10% or portion of it as your validation set. You assign dummy labels to 90% of data which doesn't have an actual label to circumvent code breaks during dls construction.

In [ ]:
path = untar_data(URLs.MNIST_TINY)
items = get_image_files(path)
tds = Datasets(items, [PILImageBW.create, [parent_label, Categorize()]], splits=GrandparentSplitter()(items))
dls = tds.dataloaders(bs=5, after_item=[ToTensor(), IntToFloatTensor()], device='cpu')

Create your model and augmentations as usual

In [ ]:
fastai_encoder = create_encoder('xresnet18', n_in=1, pretrained=False)
model = create_simclr_model(fastai_encoder, hidden_size=2048, projection_size=128)
aug_pipelines = get_simclr_aug_pipelines(size=28, rotate=False, jitter=False, bw=False, blur=False, stats=None, cuda=False)

Define self-supervised alogrithm in cbs. `ShortEpochCallback` is used for testing purposes here.

In [ ]:
cbs=[SimCLR(aug_pipelines, temp=0.07, print_augs=True)]

Pipeline: RandomResizedCrop -> RandomHorizontalFlip
Pipeline: RandomResizedCrop -> RandomHorizontalFlip


We will use `ValueMetric` from fastai since `KNNProxyMetric` is a metric implemented as a Callback. `ValueMetric` expects a function which will return a value when it's called and that function is `KNNProxyMetric.accuracy`.

In [ ]:
knn_metric_cb = KNNProxyMetric()
cbs += [knn_metric_cb]
metric = ValueMetric(knn_metric_cb.accuracy, metric_name='knn_accuracy')

Construct `Learner` with previously defined `dls`, `cbs` and `metric`.

In [ ]:
learn = Learner(dls, model, cbs=cbs, metrics=metric)

In [ ]:
learn.validate()

(#2) [1.6862106323242188,0.9871244430541992]

## Export -

In [2]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01 - augmentations.ipynb.
Converted 02 - layers.ipynb.
Converted 03 - distributed.ipynb.
Converted 10 - simclr.ipynb.
Converted 11 - moco.ipynb.
Converted 12 - byol.ipynb.
Converted 13 - swav.ipynb.
Converted 14 - barlow_twins.ipynb.
Converted 20 - clip.ipynb.
Converted 21 - clip-moco.ipynb.
Converted 70 - vision.metrics.ipynb.
Converted index.ipynb.
